In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
%cd /content/drive/MyDrive/clothes

!unzip -qq "/content/drive/MyDrive/clothes/Item-Image_02.zip"

/content/drive/MyDrive/clothes


In [ ]:
import os
from collections import Counter
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image
from glob import glob
from google.colab import drive
from keras.applications.vgg19 import VGG19
import random
import cv2
import matplotlib.gridspec as gridspec
from tensorflow.keras.optimizers import Adam  #설정해줘야함
from keras.layers import Dense , GlobalAveragePooling2D

In [ ]:
!pip3 install split-folders

In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio('/content/drive/MyDrive/clothes/Item-Image_02', output = '/content/drive/MyDrive/clothes/data', seed = 1337, ratio=(.8, .1, .1))

Copying files: 32426 files [11:21, 47.60 files/s]


In [ ]:
valid_path = '/content/drive/MyDrive/clothes/data/val'
train_path = '/content/drive/MyDrive/clothes/data/train'
test_path = '/content/drive/MyDrive/clothes/data/test'

In [ ]:
className = glob((train_path) + '/*')
number_of_class = len(className)
print(number_of_class)

4


In [ ]:
labels=pd.Series(os.listdir(train_path))

labels=labels.sort_values()
labels=labels.to_list()
labels

['bottoms', 'one-piece', 'outerwear', 'tops']

In [ ]:
numberOfClass = len(glob(train_path + "/*"))
numberOfClass

4

In [ ]:
#num of train images
len(glob(train_path + "/*/*"))

25940

In [ ]:
vgg = VGG19(include_top = False)
vgg.trainable = True    # 가지고 있는 데이터로 학습 가능 true 설정
vgg.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [ ]:
from keras.layers import Dense , GlobalAveragePooling2D

global_average_layer = GlobalAveragePooling2D()

model = Sequential([
  vgg,
  global_average_layer,
  Flatten(),
  Dense(4096,activation="relu"),
  Dense(4096,activation="relu"),
  Dense(numberOfClass, activation="softmax")
])

In [ ]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 4096)              2101248   
                                                                 
 dense_4 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_5 (Dense)             (None, 4)                 16388     
                                                      

In [ ]:
model.compile(optimizer = "rmsprop",
              loss = 'categorical_crossentropy', # 손실함수 범주형 교차엔트로피 사용
              metrics = ['accuracy']) # 지표는 정확도

In [ ]:
# 데이터셋 준비
train_datagen = ImageDataGenerator(rescale=1./255,         # 픽셀 값을 0~1 범위로 변환(scaling)
                                   rotation_range=40,      # 40도까지 회전
                                   width_shift_range=0.2,  # 20%까지 좌우 이동
                                   height_shift_range=0.2, # 20%까지 상하 이동
                                   shear_range=0.2,        # 20%까지 기울임
                                   zoom_range=0.2,         # 20%까지 확대/축소
                                   vertical_flip=True)   # 위아래 뒤집기
    
train_data = train_datagen.flow_from_directory(train_path, # 데이터가 존재하는 디렉토리
                                                    # batch_size=100, # 배치사이즈(변경가능)
                                                    class_mode='categorical', # 다중 분류이므로 categorical
                                                    color_mode= 'grayscale',
                                                    target_size=(224,224)) # 모델에 입력해줄 데이터 사이즈(변경가능)

valid_datagen = ImageDataGenerator(rescale=1./255) # 스케일링만 진행
valid_data = valid_datagen.flow_from_directory(valid_path,
                                              # batch_size=20,      
                                              class_mode='categorical',
                                              color_mode= 'grayscale', 
                                              target_size=(224, 224))

test_datagen = ImageDataGenerator(rescale=1./255) # 스케일링만 진행
test_data = test_datagen.flow_from_directory(test_path,
                                                  # batch_size=10,      
                                                  class_mode='categorical', 
                                                  color_mode= 'grayscale',
                                                  target_size=(224, 224),shuffle=False, #test data 에선  shuffle false 로 해야
                                                 save_to_dir='./test_image', save_prefix='test', save_format='jpg')

Found 25940 images belonging to 4 classes.
Found 3240 images belonging to 4 classes.
Found 3246 images belonging to 4 classes.


In [ ]:
#모델 최적화 설정
MODEL_DIR = '/content/drive/MyDrive/model_sek/' #model dir 만들기
if not os.path.exists(MODEL_DIR):
  os.mkdir(MODEL_DIR)

modelpath = '/content/drive/MyDrive/model_sek/{epoch:02d}-{val_accuracy:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath = modelpath, monitor = 'val_loss', verbose = 1, save_best_only = True)
 #valloss 기준으로 좋아지는 모델 저장
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10)
 #val loss 기준으로 더이상 안좋아지면 5까지 해보고 그만 두기

In [ ]:
# training model
hist = model.fit(train_data,
                          steps_per_epoch = 100,
                          epochs = 100,
                          validation_data = valid_data,
                          callbacks = [early_stopping_callback])

Epoch 1/100


ValueError: ignored